<h1><center><span style="color:red;">**IMPORTANT NOTICE**</span></center></h1>

Before submitting, **please**, make sure that your notebook runs **without errors** in Python 3.6
and **reproduces your solution as intended**, when you **Restart the Kernel and re-run the whole
notebook**!
<span style="color:red;">You will be severely penalized if you notebook does not run.</span>

Whereever applicable your solution will be graded based on the **plots**, generated by
**your code** on **TA's** computer.

<hr/>

# Home Assignment 4

Please, write your implementation within the designated blocks:
```python
...
### BEGIN Solution

# >>> your solution here <<<

### END Solution
...
```

Write your theoretical derivations within such blocks:
```markdown
**BEGIN Solution**

<!-- >>> your derivation here <<< -->

**END Solution**
```

## $\LaTeX$ in Jupyter
Jupyter has constantly improving $\LaTeX$ support. Below are the basic methods to
write **neat, tidy, and well typeset** equations in your notebooks:
* to write an **inline** equation use 
```markdown
$ you latex equation here $
```
* to write an equation, that is **displayed on a separate line** use 
```markdown
$$ you latex equation here $$
```
* to write a **block of equations** use 
```markdown
\begin{align}
    left-hand-side
        &= right-hand-side on line 1
        \\
        &= right-hand-side on line 2
        \\
        &= right-hand-side on the last line
\end{align}
```
The **ampersand** (`&`) aligns the equations horizontally and the **double backslash**
(`\\`) creates a new line.

<br/> <!--Intentionally left blank-->

# GLM Bayesian Models (35 pt.)

In [1]:
import numpy as np
import pandas as pd

import torch

%matplotlib inline
import matplotlib.pyplot as plt

## Task 1 (5 pt.): Theoretical problem

Consider a univariate Gaussian distribution $\mathcal{N}(x; \mu, \tau^{-1})$.
Let's define Gaussian-Gamma prior for parameters $(\mu, \tau)$:

\begin{equation}
    p(\mu, \tau)
        = \mathcal{N}(\mu; \mu_0, (\beta \tau)^{-1})
            \otimes \text{Gamma}(\tau; a, b)
        \,.
\end{equation}

Find the posterior distribution of $(\mu, \tau)$ after observing $X = (x_1, \dots, x_n)$.

**BEGIN Solution**

Likelihood: 
$$ p(X|\mu, \tau) = \prod_{i=1}^{n} \sqrt{\frac{\tau}{2\pi}} \exp\left(\frac{-\tau(x_i-\mu)}{2}\right)  = \\
=\frac{1}{(2\pi)^{n/2}}\tau^{n/2} \exp \left( -\frac{\tau}{2}\sum_{i=1}^{n} (x_i - \mu)^2 \right)=\\=
\frac{1}{(2\pi)^{n/2}}\tau^{n/2} \exp \left( - \frac{\tau}{2} \left[ \sum x_i^2 + 2\mu \sum x_i - n \mu^2\right] \right)$$

Prior:

$$ p(\mu, \tau) = NG (\mu, \tau | \mu_0, \beta, a,b)= \mathcal{N}(\mu | \mu_0, (\beta \tau)^{-1}) \otimes \text{Gamma}(\tau| a, b)=\\= \frac{1}{Z} \tau^{1/2} \exp{\left( -\frac{\beta\tau}{2}(\mu - \mu_0)^2 \right)} \tau^{a-1} \exp{\left( -b \tau \right)}$$

Posterior:

$$ p( \mu, \tau | X) \propto  p(X|\mu, \tau) p(\mu, \tau) \propto \tau^{1/2} \exp{\left( -\frac{\beta\tau}{2}(\mu - \mu_0)^2 \right)} \tau^{a-1} \exp{\left( -b \tau \right)} \times \tau^{n/2} \exp \left( - \frac{\tau}{2} \left[ \sum x_i^2 + 2\mu \sum x_i - n \mu^2\right] \right) \propto \\ \propto \tau^{1/2}  exp{ \left( -\frac{\tau}{2} \left[ \beta (\mu^2 - 2\mu \mu_0 + \mu_0^2)+\sum x_i^2 - 2\mu \sum x_i+n\mu^2 \right] \right) } \tau^{n/2+a-1} \exp{} \left( -b\tau \right)\propto \\ \propto
\tau^{1/2} exp{\left( -\frac{\tau}{2} \left[ (\beta + n)\left( \mu - \frac{\beta \mu_0 + \sum x_i}{\beta + n}\right)^2\right]\right)}\tau^{n/2+a-1} exp{\left( -A\tau \right)}$$

where $$A = b + \frac{(\beta + n)\sum x_i^2 + n\beta\mu_0^2 - \left( \sum x_i \right)^2 - 2\beta \mu_0 \sum x_i}{2(\beta+n)}$$

$ \Rightarrow $ 
$$p(\mu, \tau |X) = NG(\mu, \tau| \mu^{'} , \beta ^{'}, a^{'}, b^{'})$$,where 
$$\mu^{'} = \frac{\beta \mu_0 + \sum x_i}{\beta + n}\\
\beta ^{'} = \beta +n \\
a^{'} = a + \frac{n}{2}\\
b^{'} = A$$

PS. $\propto$ is "proportional to" (we don't care about constant because it's just normalization constant to be distribution density)

**END Solution**

<br/> <!--Intentionally left blank-->

## Task 2 (10 pt.): Practical problem

Evaluate the following integral using the Laplace approximation:
\begin{equation}
    x \mapsto \int \sigma(w^T x) \mathcal{N}(w; 0, \Sigma) dw \,,
\end{equation}
for $x = \bigl(\tfrac23, \tfrac16, \tfrac16\bigr)\in \mathbb{R}^3$ and 
\begin{equation}
    \Sigma
        = \begin{pmatrix}
             1    & -0.25 & 0.75 \\
            -0.25 &  1    & 0.5  \\
             0.75 &  0.5  & 2
           \end{pmatrix}
        \,.
\end{equation}

### Part 2.1 (4 pt.)
Use the Hessian matrix computed numericaly via finite differences. (Check out [Numdifftools](https://pypi.python.org/pypi/Numdifftools))

In [19]:
### BEGIN Solution
import numdifftools as nd
from torch.autograd import Variable
dtype = torch.FloatTensor
x = np.array([2/3,1/6,1/6]).reshape(3,1)
sigma = np.array([[1, -0.25, 0.75],
                 [-0.25, 1, 0.5],
                 [0.75, 0.5, 2]])

x_t = Variable(torch.from_numpy(x).type(dtype))
sigma_t = Variable(torch.from_numpy(sigma).type(dtype))

In [175]:
C = 1/np.sqrt((2*np.pi)**3 * np.linalg.det(sigma))
def f(omega):
    return C * np.exp(-0.5 *omega.T @ np.linalg.inv(sigma) @ omega)/(1+ np.exp(-1 * omega.T @ x))
def log_f(omega):
    return np.log(f(omega))
def f_t(z):
    return torch.exp(-0.5 * torch.mm(torch.mm(z.t(),sigma_t.inverse()),z)) * torch.sigmoid(torch.mm(z.t(), x_t))
def log_f_t(z):
    return torch.log(f_t(z))

In [176]:
#Mode of log
def gradient_accent(tolerance, max_iteration, lr):
    theta = Variable(torch.randn(3,1).type(dtype), requires_grad=True)
    
    for k in range(max_iteration):
        y = log_f_t(theta)
        y.backward()
        theta.data += lr * theta.grad.data
        if torch.norm(theta.grad.data) < tolerance:
            print('converge by tolerance')
            break
        
        theta.grad.data.zero_()
    #print(theta.data)
    return theta.data.numpy()
theta_0 = gradient_accent(1e-3, 10000, 1e-1)
print ('argmax of log =',theta_0.reshape(3,))

converge by tolerance
argmax of log = [0.32099685 0.03516578 0.39135846]


In [177]:
#hessian via nd
def hessian_nd(func, omega_0):
    return nd.Hessian(func)(omega_0)
h  = hessian_nd(log_f, theta_0.reshape(3,))

### END Solution

<br/> <!--Intentionally left blank-->

### Part 2.2 (4 pt.)
Use the diagonal approximation of the Hessian computed by autodifferentiation
in **pytorch**.

In [178]:
### BEGIN Solution

# >>> your solution here <<<
def hessian_pt(func, point):
    theta = Variable(torch.FloatTensor(point), requires_grad=True)
    y = log_f_t(theta)
    grad_params = torch.autograd.grad(y, theta, create_graph=True)
    hessian = np.diag([torch.autograd.grad(grad_params[0][i], \
                                             theta, create_graph=True)[0].data.numpy()[i][0] for i in range(3)])
    return hessian
h_t = hessian_pt(log_f_t, theta_0)


### END Solution

<br/> <!--Intentionally left blank-->

### Part 2.3 (2 pt.)

Compare the results

In [183]:
### BEGIN Solution

# >>> your solution here <<<
determinant = - np.linalg.det(h)
Z = f(theta_0)[0][0] * np.sqrt((2*np.pi)**3/determinant)
print ('Integral approximation (ND) = ', Z)

determinant_t = - np.linalg.det(h_t)
Z_t = C * f_t(Variable(torch.FloatTensor(theta_0))).data.numpy()[0][0] * np.sqrt((2*np.pi)**3/determinant_t) 
print ('Integral approximation (Torch) = ', Z_t)

### END Solution

Integral approximation (ND) =  0.49791480659914655
Integral approximation (Torch) =  0.2964293893155589


<br/> <!--Intentionally left blank-->

## Task 3 (10 pt.): Gaussian Process (theoretical)

### Part 3.1 (5 pt.)

Assuimng the matrices $A \in \mathbb{R}^{n \times n}$ and $D \in \mathbb{R}^{d \times d}$
are invertable, using **gaussian elimination** find the inverse matrix for the following
block matrix:
\begin{equation}
    \begin{pmatrix} A & B \\ C & D \end{pmatrix} \,,
\end{equation}
where $C \in \mathbb{R}^{d \times n}$ and $B \in \mathbb{R}^{n \times d}$.

**BEGIN Solution**
$$
 \left(\begin{array}{rr|rr}
  A & B & I & 0  \\
   C & D & 0 & I \\
   \end{array} \right) = 
   \left(\begin{array}{rr|rr}
  I & A^{-1}B & A^{-1} & 0  \\
   C & D & 0 & I \\
   \end{array} \right)= 
   \left(\begin{array}{rr|rr}
  I & A^{-1}B & A^{-1} & 0  \\
   C & D & 0 & I \\
   \end{array} \right)== 
   \left(\begin{array}{rr|rr}
  I & A^{-1}B & A^{-1} & 0  \\
   0 & (D - CA^{-1}B) & -CA^{-1} & I \\
   \end{array} \right)= 
   \left(\begin{array}{rr|rr}
  I & A^{-1}B & A^{-1} & 0  \\
   0 & I & -(D - CA^{-1}B)^{-1}CA^{-1} & (D - CA^{-1}B)^{-1} \\
   \end{array} \right)=
   \left(\begin{array}{rr|rr}
  I & 0 & A^{-1} + A^{-1}B(D - CA^{-1}B)^{-1}CA^{-1} & -A^{-1}B(D - CA^{-1}B)^{-1}  \\
   0 & I & -(D - CA^{-1}B)^{-1}CA^{-1} & (D - CA^{-1}B)^{-1} \\
   \end{array} \right)
$$



\begin{equation}
    \begin{pmatrix} A & B \\ C & D \end{pmatrix}^{-1}
    =
    \begin{pmatrix} A^{-1} + A^{-1}B(D - CA^{-1}B)^{-1}CA^{-1} & -A^{-1}B(D - CA^{-1}B)^{-1} \\ 
    -(D - CA^{-1}B)^{-1}CA^{-1} & (D - CA^{-1}B)^{-1} \end{pmatrix}
\end{equation}

**END Solution**

<br/> <!--Intentionally left blank-->

### Part 3.2 (5 pt.)

Assume, that the function $y(x)$, $x \in \mathbb{R}^d$, is a realization of the Gaussian
Process $GP\bigl(0; K(\cdot, \cdot)\bigr)$ with $K(a, b) = \exp({- \gamma \|a - b\|_2^2}))$.

Suppose two datasets were observed: **noiseless** and **noisy**
\begin{aligned}
    & D_0 = \bigl(x_i, y(x_i) \bigr)_{i=1}^{n} \,, \\
    & D_1 = \bigl(x'_i, y(x'_i) + \varepsilon_i \bigr)_{i=1}^{m} \,,
\end{aligned}
where $\varepsilon_i \sim \text{ iid } \mathcal{N}(0, \sigma^2)$.

Derive the conditional distribution of $y(x) \big\vert_{D_0, D_1}$ at a new $x$.

**BEGIN Solution**

<b>Lemma.</b> If $$p(x,y| \mu, \Sigma) = \mathcal{N}(\mu, \Sigma) = \mathcal{N}\left({\begin{bmatrix}\mu_x \\\mu_y\end{bmatrix}}, {\begin{bmatrix}\Sigma_{xx} & \Sigma_{xy} \\\Sigma_{yx}& \Sigma_{yy}\end{bmatrix}} \right)$$


$$
\Rightarrow p(x|y=y_0) =\mathcal{N}(\mu_x + \Sigma_{xy}\Sigma_{yy}^{-1}(y_0 - \mu_y),\Sigma_{xx}-\Sigma_{xy}\Sigma_{yy}^{-1}\Sigma_{yx}) $$

Proof: (https://people.eecs.berkeley.edu/~pabbeel/cs287-fa11/slides/Gaussians-v2.pdf) (page 10)

Then, we have (for $D_0$):

$$p(y(x),y(x_1),\dots, y(x_n)) = \mathcal{N}(0, \Sigma)$$, where
$\Sigma = \begin{pmatrix} K(x,x) & k^T \\ k & K \end{pmatrix} $

$K = \begin{pmatrix} K(x_1,x_1) &...& K(x_1, x_n) \\..&..&..\\ K(x_n,x_1) & .. & K(x_n, x_n) \end{pmatrix}, \quad k^T = \left[ K(x,x_1) \dots K(x, x_n)\right ] $


$ \Gamma = \Sigma^{-1} = [\text{by task 3.1 and Woodbury matrix identity}] =\\= 
\begin{pmatrix} K(x,x)^{-1} + K(x,x)^{-1}k^T(K - kK(x,x)^{-1}k^T)^{-1}kK(x,x)^{-1} & -K(x,x)^{-1}k^T(K - kK(x,x)^{-1}k^T)^{-1} \\ 
 -(K - kK(x,x)^{-1}k^T)^{-1}kK(x,x)^{-1} & (K - kK(x,x)^{-1}k^T)^{-1} \end{pmatrix}
$

$\Rightarrow$
$$p (y(x)| D_0) = \mathcal{N}\left(k^TK^{-1} \begin{bmatrix}y(x_1)\\.. \\y(x_n)\end{bmatrix}, K(x,x)+k^TK^{-1}k\right)$$


Let $\Sigma_1 = K(x,x)+k^TK^{-1}k$, and $\mu_1 = k^TK^{-1} \begin{bmatrix}y(x_1)\\.. \\y(x_n)\end{bmatrix}$

Then for $D_1$:

$$p(y(x),y(x_1^{'}),\dots, y(x_n^{'})) = \mathcal{N}(0, \Lambda)$$, where
$\Lambda = \begin{pmatrix} K(x,x) & k^T \\ k & K \end{pmatrix} $


**END Solution**

<br/> <!--Intentionally left blank-->

## Task 4 (10 pt.): Gaussian Process (practical)

> In the late 1950’s Charles Keeling invented an accurate way to measure atmospheric $CO_2$ concentration and began taking regular measurements at the Mauna Loa observatory.

Download the data from http://scrippsco2.ucsd.edu/data/atmospheric_co2/mlo

### Part 4.1 (3 pt.): Preprocessing

Take `monthly_in_situ_co2_mlo.csv` dataset, load it and prepare the data.

**ATTENTION** The first couple of lines of the file contain the description of
the dataset. Make sure to **read it** carefully.

Here is what the dataset might look like when loaded:
<table border="1" class="dataframe">
  <thead>
    <tr>
      <th></th>
      <th>Yr</th>
      <th>Mn</th>
      <th>Date</th>
      <th>Date</th>
      <th>CO2</th>
      <th>seasonally</th>
      <th>fit</th>
      <th>seasonally</th>
      <th>CO2</th>
      <th>seasonally</th>
    </tr>
    <tr>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th>adjusted</th>
      <th></th>
      <th>adjusted fit</th>
      <th>filled</th>
      <th>adjusted filled</th>
    </tr>
    <tr>
      <th></th>
      <th></th>
      <th></th>
      <th>Excel</th>
      <th></th>
      <th>[ppm]</th>
      <th>[ppm]</th>
      <th>[ppm]</th>
      <th>[ppm]</th>
      <th>[ppm]</th>
      <th>[ppm]</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>45</th>
      <td>1961</td>
      <td>10</td>
      <td>22569</td>
      <td>1961.7890</td>
      <td>315.31</td>
      <td>318.31</td>
      <td>314.93</td>
      <td>317.92</td>
      <td>315.31</td>
      <td>318.31</td>
    </tr>
    <tr>
      <th>142</th>
      <td>1969</td>
      <td>11</td>
      <td>25522</td>
      <td>1969.8740</td>
      <td>322.85</td>
      <td>324.80</td>
      <td>323.15</td>
      <td>325.08</td>
      <td>322.85</td>
      <td>324.80</td>
    </tr>
    <tr>
      <th>168</th>
      <td>1972</td>
      <td>1</td>
      <td>26313</td>
      <td>1972.0410</td>
      <td>326.77</td>
      <td>326.74</td>
      <td>326.82</td>
      <td>326.78</td>
      <td>326.77</td>
      <td>326.74</td>
    </tr>
    <tr>
      <th>179</th>
      <td>1972</td>
      <td>12</td>
      <td>26648</td>
      <td>1972.9563</td>
      <td>327.55</td>
      <td>328.36</td>
      <td>327.71</td>
      <td>328.51</td>
      <td>327.55</td>
      <td>328.36</td>
    </tr>
    <tr>
      <th>249</th>
      <td>1978</td>
      <td>10</td>
      <td>28778</td>
      <td>1978.7890</td>
      <td>332.55</td>
      <td>335.74</td>
      <td>332.71</td>
      <td>335.89</td>
      <td>332.55</td>
      <td>335.74</td>
    </tr>
  </tbody>
</table>

Your task:
0. Load the `CO2 [ppm]` time series
1. Replace $-99.99$ with **NaN** and drop the missing observations
2. Construct a *decimal years time index*: **January** of the year of the first observation must correspond to $0.0$
3. Split the time series into train and test
4. Normalize the target value by fitting a transformation on the train
5. Plot the resulting target against the time index

In [510]:
### BEGIN Solution

# >>> your solution here <<<
import pandas as pd
df = pd.read_csv("monthly_in_situ_co2_mlo.csv", index_col=1, header=54)
df.replace(to_replace=-99.99, value=np.nan, inplace=True)
df.head()
### END Solution

Yr      Date        Date        CO2  seasonally          fit  \
 Mn                                                                   
                                              adjusted                
              Excel                  [ppm]      [ppm]         [ppm]   
 01  1958     21200   1958.0411     -99.99      -99.99       -99.99   
 02  1958     21231   1958.1260     -99.99      -99.99       -99.99   
 03  1958     21259   1958.2027     315.70      314.43       316.18   

       seasonally           CO2       seasonally  
 Mn                                               
     adjusted fit        filled  adjusted filled  
            [ppm]         [ppm]            [ppm]  
 01        -99.99        -99.99           -99.99  
 02        -99.99        -99.99           -99.99  
 03        314.90        315.70           314.43

<br/> <!--Intentionally left blank-->

### Part 4.2 (7 pt.)

Use [**GPy**](https://pypi.python.org/pypi/GPy) library for training and prediction.
Make sure you've read the documentation!. Consult with the seminar if necessary.

Your general task is:
* to fit a GP and run the predict on the test:
    * useful kernels to combine: `GPy.kern.RBF`, `GPy.kern.Poly`, `GPy.kern.StdPeriodic`, `GPy.kern.White`
* to plot residuals. Inspect them on normality by QQ-plot, try to make them as "normal" as possible
* to provide a point-estimate of $R^2$ score on the test part

In [ ]:
### BEGIN Solution

# >>> your solution here <<<

### END Solution

<br/> <!--Intentionally left blank-->